In [1]:
with open('data/wikitionary_data/raw_data_stage_03.json', 'r', encoding="utf8") as f:
    all_lines = f.readlines()

In [2]:
import json
raw_download_links_list = []
for n, line in enumerate(all_lines):
    dict_line = json.loads(line)
    for sound in dict_line["s"]:
        if "audio" in sound:
            raw_download_links_list.append([int(sound["audio_id"]), sound["audio"]])

In [3]:
with open('data/audio_data/!ids.json', 'w', encoding="utf8") as f:
    for raw_link in raw_download_links_list:
        # f.write(json.dumps(datum, ensure_ascii=False)+"\n")
        f.write(str(raw_link[0])+"\t"+raw_link[1]+"\n")

In [4]:
with open('data/audio_data/!ids.json', 'r', encoding="utf8") as f:
    str_raw_download_links_list = f.readlines()

raw_download_links_list = []
for str_raw_line in str_raw_download_links_list:
    str_raw_line_sep = str_raw_line[:-1].split("\t")
    raw_download_links_list.append([int(str_raw_line_sep[0]), str_raw_line_sep[1]])

In [5]:
# Проверка
with open('data/audio_data/!ids.json', 'r', encoding="utf8") as f:
    new_str = ""
    for raw_link in raw_download_links_list:
        new_str += str(raw_link[0])+"\t"+raw_link[1]+"\n"
    old_str = f.read()
# old_str == new_str

In [6]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

# Функция загрузки и сохранения аудио
def download_wikimedia(raw_link):
    raw_link[1] = raw_link[1].replace("&#39;", "'")
    raw_link[1] = raw_link[1].replace("%2C;", ",")
    first_link = "https://commons.wikimedia.org/wiki/File:"+urllib.parse.quote(raw_link[1])
    response = requests.get(first_link,
                            headers={'User-Agent': 'enWiktionary_download_audios/0.0 (example@mail.ru)'})
    response_content = response.content.decode("utf-8")

    soup = BeautifulSoup(response_content, "html.parser")
    download_link = soup.find("a", attrs={"aria-label":"Download file"})["href"]
    download_link = str(download_link)
    if not download_link.startswith("https://upload.wikimedia.org/"):
        0/0
    else:
        response = requests.get(download_link.split("?")[0],
                                headers={'User-Agent': 'enWiktionary_download_audios/0.0 (example@mail.ru)'})
        response_content = response.content
        
        if response_content[:9] == b'<!DOCTYPE':
            0/0
        elif response_content[:9] == b'File not ':
            0/0
        else:
            open("data/audio_data/"+str(raw_link[0])+".mp3", "wb").write(response_content)

In [10]:
import time
import threading
import random
import os

# Мультипотоковая загрузка аудио

def do_files():
    files = os.listdir("data/audio_data/")
    files.remove("!ids.json")
    files = set([int(i.split(".")[0]) for i in files])
    return files

download_links_list = []

start_with = "" # raw_download_links_list[19250][1]
links_count = len(raw_download_links_list)
print(links_count)
exclude_files = [54973]

threads = []

def download_with_retry(raw_link):
    while True:
        try:
            download_wikimedia(raw_link)
            print(f"===Done {raw_link}===")
            break
        except Exception as e:
            # print()
            # print(e)
            period = random.randint(0, 2)
            print(f"===Waiting {period}s for {raw_link}===")
            break
            # print()
            time.sleep(period)

def all_download(raw_download_links_list, start_with, files, threads):
    to_start = False
    threads_limit = 800
    count = 0
    for raw_link in raw_download_links_list:
        if raw_link[1] == start_with or start_with == "":
            to_start = True
        if to_start and raw_link[0] not in files and count < threads_limit and raw_link[0] not in exclude_files:
            count += 1
            thread = threading.Thread(target=download_with_retry, args=(raw_link,))
            thread.start()
            threads.append(thread)
    return threads

def all_threads(raw_download_links_list, start_with, files, threads):
    threads = all_download(raw_download_links_list, start_with, files, threads)

    for thread in threads:
        thread.join()
    return threads

def all_threads_finished(threads):
    for thread in threads:
        if thread.is_alive():
            return False
    return True

files = do_files()
is_starting = True
while True:
    if (len(files)+len(exclude_files)) == links_count:
        break
    while True:
        if is_starting:
            files = do_files()
            print(f"==Progress {((len(files)+len(exclude_files))/links_count)*100}==")
            threads = all_threads(raw_download_links_list, start_with, files, threads)
            is_starting = False
        else:
            if all_threads_finished(threads):
                is_starting = True
                break
    


260142


In [8]:
from urllib.parse import unquote

unquote("LL-Q14196%20%28afr%29-Oesjaar-oranje.wav")

'LL-Q14196 (afr)-Oesjaar-oranje.wav'

In [9]:
import urllib.parse

urllib.parse.quote("???")

'%3F%3F%3F'